#### Read in data

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('msfa_data.csv')
df['obs_se'] = .05
df['group'] = np.arange(df.shape[0])
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'msfa_data.csv'

#### Build inputs

In [ ]:
from anml.parameter.parameter import Parameter, ParameterSet
from anml.parameter.prior import GaussianPrior, Prior
from anml.parameter.processors import process_all
from anml.parameter.spline_variable import Spline, SplineLinearConstr
from anml.parameter.variables import Intercept
from sfma.data import DataSpecs
from sfma.data import Data

In [ ]:
data_spec = DataSpecs(col_obs='uhc', col_obs_se='obs_se')

increasing = SplineLinearConstr(order=1, y_bounds=[0.0, np.inf], grid_size=20)
concave = SplineLinearConstr(order=2, y_bounds=[-np.inf, 0.0], grid_size=20)                                                

spline = Spline(
    covariate='physicians',
    knots_type='domain', knots_num=4, 
    degree=3, r_linear=True, include_intercept=True, 
    derivative_constr=[increasing, concave],
)
spline.set_fe_prior(Prior(lower_bound=[0.0] * spline.num_fe, upper_bound=[1.0] * spline.num_fe))
param_set_marginal = ParameterSet([Parameter(param_name='beta', variables=[spline])])
process_all(param_set_marginal, df)


intercept = Intercept(add_re=True, col_group='group', re_prior=GaussianPrior(lower_bound=[0.0], upper_bound=[np.inf]))
param_set_v = ParameterSet([Parameter(param_name='v', variables=[intercept])])
process_all(param_set_v, df)

data = Data(data_spec)
data.process(df)

#### Build Models

In [ ]:
from sfma.models.maximal import VModel, UModel
from sfma.models.marginal import SimpleBetaEtaModel

In [ ]:
marginal_model = SimpleBetaEtaModel(param_set_marginal)
v_model = VModel(param_set_v) 

#### Solve

In [ ]:
from anml.solvers.base import ClosedFormSolver, ScipyOpt, IPOPTSolver

In [ ]:
marginal_solver = IPOPTSolver(marginal_model)
v_solver = ClosedFormSolver(v_model)

In [ ]:
from sfma.solver import SimpleSolver

In [ ]:
ss = SimpleSolver([marginal_solver, v_solver])

In [ ]:
x_init = np.random.randn(marginal_solver.model.x_dim)
ss.fit(x_init, data)


#### Plot

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(df['physicians'].to_numpy(), data.obs, '.', label='data')
plt.plot(df['physicians'].to_numpy(), data.obs + ss.x_opt[1], '.', label='data + v')
plt.plot(df['physicians'].to_numpy(), ss.predict(), '.r', label='x*beta')